In [2]:
import sys
sys.path

['/Users/litos/opt/anaconda3/envs/tensorflow_cpu/lib/python36.zip',
 '/Users/litos/opt/anaconda3/envs/tensorflow_cpu/lib/python3.6',
 '/Users/litos/opt/anaconda3/envs/tensorflow_cpu/lib/python3.6/lib-dynload',
 '',
 '/Users/litos/opt/anaconda3/envs/tensorflow_cpu/lib/python3.6/site-packages',
 '/Users/litos/opt/anaconda3/envs/tensorflow_cpu/lib/python3.6/site-packages/IPython/extensions',
 '/Users/litos/.ipython']

In [3]:
from gurobipy import *

from scipy.sparse import csr_matrix, load_npz, save_npz
from tqdm import tqdm
from sklearn.preprocessing import normalize
from collections import *
import datetime
import json
import pandas as pd
import time
# import yaml
import scipy.sparse as sparse
from ast import literal_eval

import numpy as np
import matplotlib.pyplot as plt
import random

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import normalize
from sklearn.linear_model import Ridge
from sklearn.utils.extmath import randomized_svd

from scipy.optimize import minimize

# Utils

In [4]:
def save_dataframe_csv(df, path, name):
    df.to_csv(path+name, index=False)


def load_dataframe_csv(path, name, index_col=None):
    return pd.read_csv(path+name, index_col=index_col)


# Load Data

In [5]:
# Load Original Data
df_train = pd.read_csv('../../data/yelp/Train.csv',encoding='latin-1')
# df_valid = pd.read_csv('../../data/yelp/Valid.csv',encoding='latin-1')
# df_test = pd.read_csv('../../data/yelp/Test.csv',encoding='latin-1')

In [6]:
keyphrases = pd.read_csv('../../data/yelp/KeyPhrases.csv')['Phrases'].tolist()
keyphrase_popularity = np.loadtxt('../data/yelp/'+'keyphrase_popularity.txt', dtype=int)

# Load U-I Data 
rtrain = load_npz("../../data/yelp/Rtrain.npz")
rvalid = load_npz("../../data/yelp/Rvalid.npz")
rtest = load_npz("../../data/yelp/Rtest.npz")

# Load user/item keyphrase data
U_K = load_npz("../../data/yelp/U_K.npz")
I_K = load_npz("../../data/yelp/I_K.npz")

# Models

In [7]:
def get_I_K(df, row_name = 'ItemIndex', shape = (3668,75)):
    rows = []
    cols = []
    vals = []
    for i in tqdm(range(df.shape[0])):
        key_vector = literal_eval(df['keyVector'][i])
        rows.extend([df[row_name][i]]*len(key_vector)) ## Item index
        cols.extend(key_vector) ## Keyword Index
        vals.extend(np.array([1]*len(key_vector)))
    return csr_matrix((vals, (rows, cols)), shape=shape)


In [8]:
# PLREC 
def inhour(elapsed):
    return time.strftime('%H:%M:%S', time.gmtime(elapsed))

def plrec(matrix_train, iteration=4, lamb=80, rank=200, seed=1):
    """
    Function used to achieve generalized projected lrec w/o item-attribute embedding
    :param matrix_train: user-item matrix with shape m*n
    :param iteration: number of power iterations in randomized svd
    :param lamb: parameter of penalty
    :param rank: latent dimension size
    :param seed: the seed of the pseudo random number generator to use when shuffling the data
    :return: prediction in sparse matrix
    """
    print ("Randomized SVD")
    start_time = time.time()
    P, sigma, Qt = randomized_svd(matrix_train,
                                  n_components=rank,
                                  n_iter=iteration,
                                  random_state=seed)

    RQ = matrix_train.dot(sparse.csc_matrix(Qt.T*np.sqrt(sigma)))

    print("Elapsed: {}".format(inhour(time.time() - start_time)))

    print ("Closed-Form Linear Optimization")
    start_time = time.time()
    pre_inv = RQ.T.dot(RQ) + lamb * sparse.identity(rank, dtype=np.float32)
    inverse = sparse.linalg.inv(pre_inv.tocsc())
    Y = inverse.dot(RQ.T).dot(matrix_train)
    print("Elapsed: {}".format(inhour(time.time() - start_time)))

    return np.array(RQ.todense()), np.array(Y.todense()), None

# def predict_vector(rating_vector, train_vector, remove_train=True):
#     dim = len(rating_vector)
#     candidate_index = np.argpartition(-rating_vector, dim-1)[:dim]
#     prediction_items = candidate_index[rating_vector[candidate_index].argsort()[::-1]]
    
#     if remove_train:
#         return np.delete(prediction_items, np.isin(prediction_items, train_vector.nonzero()[1]).nonzero()[0])
#     else:
#         return prediction_items

    
def predict_scores(matrix_U, matrix_V, bias=None,
                   penalize = False,
                   keyphrase_freq = I_K, 
                   critiqued_keyphrase = 0, 
                   matrix_Train = rtrain,
                   alpha = 0):
    prediction = matrix_U.dot(matrix_V.T)
    # Penalize
    if penalize == True:
        items_with_keyphrase = np.ravel(keyphrase_freq.T[critiqued_keyphrase].nonzero()[1])
        items_without_keyphrase = np.setdiff1d(np.arange(matrix_Train.shape[1]), items_with_keyphrase)
        prediction[items_without_keyphrase] = alpha # penalize
    
    return prediction

def predict_vector(rating_vector, train_vector, remove_train=True):
    dim = len(rating_vector)
    candidate_index = np.argpartition(-rating_vector, dim-1)[:dim]
    prediction_items = candidate_index[rating_vector[candidate_index].argsort()[::-1]]
    
    if remove_train:
        return np.delete(prediction_items, np.isin(prediction_items, train_vector.nonzero()[1]).nonzero()[0])
    else:
        return prediction_items


In [9]:
# initial Prediction
def predict_scores(matrix_U, matrix_V, bias=None,
                   penalize = False,
                   keyphrase_freq = I_K, 
                   critiqued_keyphrase = 0, 
                   matrix_Train = rtrain,
                   alpha = 0):
    
    prediction = matrix_U.dot(matrix_V.T)
    # Penalize
    if penalize == True:
        items_with_keyphrase = np.ravel(keyphrase_freq.T[critiqued_keyphrase].nonzero()[1])
        items_without_keyphrase = np.setdiff1d(np.arange(matrix_Train.shape[1]), items_with_keyphrase)
        prediction[items_without_keyphrase] = alpha # penalize
    
    return prediction


In [10]:
# Keyphrase Selection Helpers
def get_valid_keyphrases(keyphrase_freq,top_recommendations,item = None,threshold=50,mutiple_keyphrases_en = False, top_items = None):
    """
    Wrapper function to get either top 1 or top n keyphrases
    """
    if mutiple_keyphrases_en:
        top_keyphrases = []
        for item in top_items:
            top_keyphrases.extend(get_valid_keyphrases_for_one_item(keyphrase_freq,top_recommendations,item,threshold=threshold))
        return np.ravel(list(set(top_keyphrases))) # remove duplicate and reformat to np array
    else:
        return get_valid_keyphrases_for_one_item(keyphrase_freq,top_recommendations,item,threshold=threshold)

def get_valid_keyphrases_for_one_item(keyphrase_freq,top_recommendations, item,threshold=50):
    """
    Get keyphrases of item that make sense
    E.g. if the item has fewer than threshold=50 keyphrases, get all of them
    otherwise get top 50 keyphrases
    """
    keyphrase_length = len(keyphrase_freq[item].nonzero()[1])
    if keyphrase_length<threshold:
        return keyphrase_freq[item].nonzero()[1]
    else:
        keyphrases = np.ravel(keyphrase_freq[top_recommendations[0]].todense())
        top_keyphrases = np.argsort(keyphrases)[::-1][:threshold]
        return top_keyphrases
    
# For keyphrase selecting method # 3 "diff" 
def get_item_keyphrase_freq(keyphrase_freq,item):
    """
    Get item's keyphrase frequency 
    """
    count = keyphrase_freq[item].todense()
    return np.ravel(count/(np.sum(count)+0.001))

def get_all_item_keyphrase_freq(item_keyphrase_freq = I_K):
    res = []
    num_items = item_keyphrase_freq.shape[0]
    for item in range(num_items):
        res.append(np.ravel(get_item_keyphrase_freq(item_keyphrase_freq,item)))
    return np.array(res)

def get_keyphrase_popularity(df,keyphrases):
    """
    Get keyphrase popularity (count) from dataframe
    """
    keyphrase_popularity = np.zeros(len(keyphrases)) #initialize
    for i in range(len(df)):
        keyphrase_vector = literal_eval(df['keyVector'][i])
        keyphrase_popularity[keyphrase_vector] += 1 # count
    return keyphrase_popularity

In [11]:
all_item_keyphrase_freq = get_all_item_keyphrase_freq()

In [12]:
# One hot encoding of critiquing
def get_critiqued_UK(user_keyphrase_frequency,user_index,critiqued_keyphrase):
    """
    user_keyphrase_frequency is the U_K matrix (csr sparse matrix)
    return the one-hot encoding of the critique
    """
    U_K_cp = user_keyphrase_frequency.copy()
    U_K_cp[user_index] = 0
    U_K_cp[user_index,critiqued_keyphrase] = 1
    return U_K_cp

def project_one_hot_encoding(reg, user_keyphrase_frequency,user_index = 0,critiqued_keyphrase = 0, normalize_en = True):
    """
    Return the projection on user_sim space from one-hot encoding of critiqued keyphrase
    The res[user_index] should be target embedding row
    """
    critiqued_matrix = get_critiqued_UK(user_keyphrase_frequency, user_index, critiqued_keyphrase)
    res = reg.predict(critiqued_matrix)
    if normalize_en:
        res = normalize((res))
    return res

In [13]:
# Upper bound method 
def get_all_affected_items(wanted_keyphrases,keyphrase_freq):
    res = []
    for keyphrase in wanted_keyphrases:
        items = np.ravel(keyphrase_freq.T[keyphrase].nonzero()[1])
        res.extend(items)
    return np.array(list(set(res)))
    
def select_only_wanted_keyphrase(top_recommendations, wanted_keyphrases, keyphrase_freq, matrix_Train = rtrain):
    all_items_with_keyphrases = get_all_affected_items(wanted_keyphrases,keyphrase_freq)
    affected_items = np.setdiff1d(np.arange(matrix_Train.shape[1]), all_items_with_keyphrases) # Get all other keyphrases
    top_recommendations[~np.in1d(top_recommendations, affected_items)]
    return top_recommendations

def pruning(prediction_score, 
           wanted_keyphrases_random, 
           top_recommendations, 
           keyphrase_freq, 
           matrix_Train = rtrain,
           alpha = 0):
    items_with_keyphrase = get_all_affected_items(wanted_keyphrases_random, keyphrase_freq)
    #Return the unique values in ar1 that are not in ar2.
    items_without_keyphrase = np.setdiff1d(np.arange(matrix_Train.shape[1]), items_with_keyphrase)
#     print (items_without_keyphrase)
    print (sum(prediction_score[items_without_keyphrase]))
    score = np.copy(prediction_score)
    score[items_without_keyphrase] = alpha # penalize
    return score

# Utils

In [14]:
# Utility function for getting restaurant info from ItemIndex
def get_business_df(path = "../../data/yelp/business.json" ):
    with open(path,encoding="utf8") as json_file:
        data = json_file.readlines()
        data = list(map(json.loads, data))
    df = pd.DataFrame(data)
    
    return df

def get_restaurant_info(business_df, business_id, name = True, review_count = True, stars = True ):
    output_list = {}
    row_idx = int(business_df.index[business_df['business_id'] == business_id].tolist()[0])
    if name == True:
        output_list['name'] = business_df['name'][row_idx].encode('utf-8').strip()
    if review_count == True:
        output_list['review_count'] = business_df['review_count'][row_idx]
    if stars == True:
        output_list['stars'] = business_df['stars'][row_idx] 
    return output_list

# def get_businessid_from_Itemindex(ItemIndex_list, itemindex):
#     return ItemIndex_list['business_id'].tolist()[itemindex]

def get_restaurant_name(df_train, business_df, ItemIndex):
    rows = np.where(df_train['ItemIndex'] == ItemIndex)
    if len(rows)!= 0:
        business_id = df_train.loc[rows[0][0]]['business_id']
        item_info = get_restaurant_info(business_df, business_id)
        return item_info['name']
    return "NOT_FOUND"

# Evaluation 

In [15]:
# Evluation 
def recallk(vector_true_dense, hits, **unused):
    hits = len(hits.nonzero()[0])
    return float(hits)/len(vector_true_dense)

def precisionk(vector_predict, hits, **unused):
    hits = len(hits.nonzero()[0])
    return float(hits)/len(vector_predict)


def average_precisionk(vector_predict, hits, **unused):
    precisions = np.cumsum(hits, dtype=np.float32)/range(1, len(vector_predict)+1)
    return np.mean(precisions)


def r_precision(vector_true_dense, vector_predict, **unused):
    vector_predict_short = vector_predict[:len(vector_true_dense)]
    hits = len(np.isin(vector_predict_short, vector_true_dense).nonzero()[0])
    return float(hits)/len(vector_true_dense)


def _dcg_support(size):
    arr = np.arange(1, size+1)+1
    return 1./np.log2(arr)


def ndcg(vector_true_dense, vector_predict, hits):
    idcg = np.sum(_dcg_support(len(vector_true_dense)))
    dcg_base = _dcg_support(len(vector_predict))
    dcg_base[np.logical_not(hits)] = 0
    dcg = np.sum(dcg_base)
    return dcg/idcg


def click(hits, **unused):
    first_hit = next((i for i, x in enumerate(hits) if x), None)
    if first_hit is None:
        return 5
    else:
        return first_hit/10


def evaluate(matrix_Predict, matrix_Test, metric_names =['R-Precision', 'NDCG', 'Precision', 'Recall', 'MAP'], atK = [5, 10, 15, 20, 50], analytical=False):
    """
    :param matrix_U: Latent representations of users, for LRecs it is RQ, for ALSs it is U
    :param matrix_V: Latent representations of items, for LRecs it is Q, for ALSs it is V
    :param matrix_Train: Rating matrix for training, features.
    :param matrix_Test: Rating matrix for evaluation, true labels.
    :param k: Top K retrieval
    :param metric_names: Evaluation metrics
    :return:
    """
    global_metrics = {
        "R-Precision": r_precision,
        "NDCG": ndcg,
        "Clicks": click
    }

    local_metrics = {
        "Precision": precisionk,
        "Recall": recallk,
        "MAP": average_precisionk
    }

    output = dict()

    num_users = matrix_Predict.shape[0]

    for k in atK:

        local_metric_names = list(set(metric_names).intersection(local_metrics.keys()))
        results = {name: [] for name in local_metric_names}
        topK_Predict = matrix_Predict[:, :k]

        for user_index in tqdm(range(topK_Predict.shape[0])):
            vector_predict = topK_Predict[user_index]
            if len(vector_predict.nonzero()[0]) > 0:
                vector_true = matrix_Test[user_index]
                vector_true_dense = vector_true.nonzero()[1]
                hits = np.isin(vector_predict, vector_true_dense)

                if vector_true_dense.size > 0:
                    for name in local_metric_names:
                        results[name].append(local_metrics[name](vector_true_dense=vector_true_dense,
                                                                 vector_predict=vector_predict,
                                                                 hits=hits))

        results_summary = dict()
        if analytical:
            for name in local_metric_names:
                results_summary['{0}@{1}'.format(name, k)] = results[name]
        else:
            for name in local_metric_names:
                results_summary['{0}@{1}'.format(name, k)] = (np.average(results[name]),
                                                              1.96*np.std(results[name])/np.sqrt(num_users))
        output.update(results_summary)

    global_metric_names = list(set(metric_names).intersection(global_metrics.keys()))
    results = {name: [] for name in global_metric_names}

    topK_Predict = matrix_Predict[:]

    for user_index in tqdm(range(topK_Predict.shape[0])):
        vector_predict = topK_Predict[user_index]

        if len(vector_predict.nonzero()[0]) > 0:
            vector_true = matrix_Test[user_index]
            vector_true_dense = vector_true.nonzero()[1]
            hits = np.isin(vector_predict, vector_true_dense)

            # if user_index == 1:
            #     import ipdb;
            #     ipdb.set_trace()

            if vector_true_dense.size > 0:
                for name in global_metric_names:
                    results[name].append(global_metrics[name](vector_true_dense=vector_true_dense,
                                                              vector_predict=vector_predict,
                                                              hits=hits))

    results_summary = dict()
    if analytical:
        for name in global_metric_names:
            results_summary[name] = results[name]
    else:
        for name in global_metric_names:
            results_summary[name] = (np.average(results[name]), 1.96*np.std(results[name])/np.sqrt(num_users))
    output.update(results_summary)

    return output



# Critiquing Pipline

In [16]:
business_df = get_business_df()

In [17]:
keyphrase_popularity = np.loadtxt('../data/yelp/'+'keyphrase_popularity.txt', dtype=int)

In [18]:
Y, RQt, Bias = plrec(rtrain,
                    iteration = 10,
                    lamb = 200,
                    rank = 200)
RQ = RQt.T
reg = LinearRegression().fit(normalize(U_K), Y)

Randomized SVD
Elapsed: 00:00:00
Closed-Form Linear Optimization
Elapsed: 00:00:00


In [19]:
# Set up dataframe 

# post_ranki is post rank with different lambda ratio for combining pre-post User similarity matrix 

columns = ['user_id', 'target_item', 'item_name', 'iter', 'pre_rank', 
           'top_prediction_item_name',
           'post_rank_random_all',
           'post_rank_random_upper',
           'random_scores',
           'post_rank_pop_all',
           'post_rank_pop_upper',
           'pop_scores',
           'post_rank_diff_all',
           'post_rank_diff_upper',
           'diff_scores',
           'critiqued_keyphrase_random',
           'keyphrase_name_random',
           'critiqued_keyphrase_pop',
           'keyphrase_name_pop',
           'critiqued_keyphrase_diff',
           'keyphrase_name_diff',
           'num_existing_keyphrases',
           'pure_pruning_rank'] 
df = pd.DataFrame(columns=columns)
row = {}

## LP Objectives

In [28]:
def Average(initial_prediction_u, keyphrase_freq, affected_items, unaffected_items, num_keyphrases, query, test_user, item_latent, reg, all_equal = True):
    critiqued_vector = np.zeros(keyphrase_freq.shape[1])
    
    for q in query:
#         critiqued_vector[q] = 1
        critiqued_vector[q] = max(keyphrase_freq[test_user , q],1)
        
    num_critiques = len(query)
    
    # Get item latent for updating prediction
    W2 = reg.coef_
    W = item_latent.dot(W2)
    
    optimal_lambda = 1 # weight all critiquing equally
    
    lambdas = [optimal_lambda]*num_critiques
    
    # Record lambda values 
    for k in range(num_critiques):
        critiqued_vector[query[k]] *= optimal_lambda

    # Get rating score
    critique_score = predict_scores(matrix_U=reg.predict(critiqued_vector.reshape(1, -1)),
                                    matrix_V=item_latent)
    new_prediction = initial_prediction_u + critique_score.flatten()
    
#     if all_equal:
#         # weight initial and each critiquing equally 
#         new_prediction = initial_prediction_u/(num_critiques) + critique_score.flatten()
#     else:
#         # weight intial and combined critiquing equally
#         new_prediction = initial_prediction_u + critique_score.flatten() 
# #     print (len(new_prediction))
    return new_prediction, lambdas   


## RankSVM Objective

In [33]:
#### See https://www.overleaf.com/read/wwftdhpcmxnx
#### For the RankSVM math

def rankSVM(initial_prediction_u, keyphrase_freq, affected_items, unaffected_items, num_keyphrases, 
            query, test_user, item_latent, reg, user_latent_embedding, item_keyphrase_freq = I_K):
    critiques = query # fix this variable name later
    
#     critiqued_vector = np.zeros(keyphrase_freq.shape[1])
    
#     for c in critiques:
# #         critiqued_vector[q] = 1 # set critiqued/boosted keyphrase to 1
#         critiqued_vector[c] = max(keyphrase_freq[test_user , c],1)
# #         print ('critiqued_vector setting: ',critiqued_vector[q])
#     print ('affected items, ',affected_items)
#     print ('unaffected items, ',unaffected_items)
    
    num_critiques = len(critiques)

    W2 = reg.coef_
    W = item_latent.dot(W2)

    num_affected_items = len(affected_items)
    num_unaffected_items = len(unaffected_items)

    start_time = time.time()

    # Model
    m = Model("LP2RankSVM")
    m.setParam('OutputFlag', 0)
    
    # Assignment variables
    thetas = []
    us = []
    xis = []
    # weight thetas
    for k in range(num_critiques + 1):
        thetas.append(m.addVar(lb=-1,
                              ub=1,
                              vtype=GRB.CONTINUOUS,
                              name="theta%d" % k))
    thetas = np.array(thetas)
    # dummy variable u for absolute theta
    for k in range(num_critiques + 1):
        us.append(m.addVar(vtype=GRB.CONTINUOUS,
                          name="u%d" % k))
        
    # slack variables xi
    for i in range(num_affected_items):
        for j in range(num_unaffected_items):
            xis.append(m.addVar(lb = 0, 
                                vtype = GRB.CONTINUOUS,
                                name = "xi_%d_%d" % (i,j) ))
      
    ## constraints
    # constraints for dummy variable u's
    for k in range(num_critiques+1):
        m.addConstr(us[k] >= thetas[k])
        m.addConstr(us[k] >= -thetas[k])
        
    
    ## Pre-calculate critique embedding
    u_i = Y[test_user]
    phi_js = []
    phi_jprimes = []
    k_cis = []
    
    user_latent_embedding = np.array(user_latent_embedding)
#     print ('user latent embedding shape: ', user_latent_embedding.shape)
    # constraints for rankSVM 
    for j in range(num_affected_items):
        for j_ in range(num_unaffected_items):
            m.addConstr( thetas.dot(user_latent_embedding.dot(RQ[affected_items[j]])) >= thetas.dot(user_latent_embedding.dot(RQ[unaffected_items[j_]])) + 1 - xis[j*num_affected_items + j_], name = "constraints%d_%d" % (j,j_))
#             print ('item j embedding :',user_latent_embedding.dot(RQ[j]) )
#             print ('item j_ embedding:',user_latent_embedding.dot(RQ[j_]) )
    lamb = 5 #regularization parameter (trading-off margin size against training error
    m.setObjective(quicksum(us) + lamb * quicksum(xis), GRB.MINIMIZE)
                
    # Optimize
    m.optimize()

#     print("Elapsed: {}".format(inhour(time.time() - start_time)))

    thetas = []
    for k in range(num_critiques+1):
        optimal_theta = m.getVarByName("theta%d" % k).X
        thetas.append(optimal_theta)
        
#     print ('optimal thetas: ',thetas)


    critiqued_vector = np.zeros(keyphrase_freq.shape[1])
    
    # Combine weights to critiqued vector
    for c in critiques:
#         critiqued_vector[q] = 1 # set critiqued/boosted keyphrase to 1
        critiqued_vector[c] = max(keyphrase_freq[test_user , c],1)
    for k in range(num_critiques):
        critiqued_vector[critiques[k]] *= thetas[k+1]
    
    # Get rating score
    critique_score = predict_scores(matrix_U=reg.predict(critiqued_vector.reshape(1, -1)),
                                    matrix_V=item_latent)
    new_prediction = thetas[0]*initial_prediction_u + critique_score.flatten()
    
    return new_prediction, thetas

In [41]:
#### See https://www.overleaf.com/read/wwftdhpcmxnx
#### For the RankSVM math

def rankSVM2(initial_prediction_u, keyphrase_freq, affected_items, unaffected_items, num_keyphrases, 
            query, test_user, item_latent, reg, user_latent_embedding, item_keyphrase_freq = I_K, lamb = [5,5]):
    critiques = query # fix this variable name later

    # pre calculate some value
    num_critiques = len(critiques)

    num_affected_items = len(affected_items)
    num_unaffected_items = len(unaffected_items)

#     start_time = time.time()

    # Model
    m = Model("LP2RankSVM2")
    m.setParam('OutputFlag', 0) # set to 1 for outputing details
    
    # Assignment variables
    thetas = []
    us = []
    xi_pos = []
    xi_neg = []
    # weight thetas
    for k in range(num_critiques + 1):
        thetas.append(m.addVar(lb=-1,
                              ub=1,
                              vtype=GRB.CONTINUOUS,
                              name="theta%d" % k))
    thetas = np.array(thetas)
    
    # dummy variable u for absolute theta
    for k in range(num_critiques + 1):
        us.append(m.addVar(vtype=GRB.CONTINUOUS,
                          name="u%d" % k))
        
    # slack variables xi
    for i in range(num_affected_items):
        xi_pos.append(m.addVar(lb = 0, 
                                vtype = GRB.CONTINUOUS,
                                name = "xi_pos%d" % i ))
    for i in range(num_unaffected_items):
        xi_neg.append(m.addVar(lb = 0, 
                                vtype = GRB.CONTINUOUS,
                                name = "xi_neg%d" % i ))
        
    ## constraints
    # constraints for dummy variable u's
    for k in range(num_critiques+1):
        m.addConstr(us[k] >= thetas[k])
        m.addConstr(us[k] >= -thetas[k])
 
    user_latent_embedding = np.array(user_latent_embedding)
    
    # Affected items rank higher
    for j in range(num_affected_items):
        m.addConstr( thetas.dot(user_latent_embedding.dot(RQ[affected_items[j]])) >= initial_prediction_u[affected_items[j]] + 1 - xi_pos[j], name = "pos_constraint%d" % j )
    
    # Unaffected items rank lower
    for j in range(num_unaffected_items):
        m.addConstr( initial_prediction_u[unaffected_items[j]] - thetas.dot(user_latent_embedding.dot(RQ[unaffected_items[j]])) >=  1 - xi_neg[j], name = "neg_constraint%d" % j )
            
    # objective
    lamb1 = lamb[0] #regularization for trading-off margin size against training error
    lamb2 = lamb[1] #regularization for trading-off deviation from Averaging 

    m.setObjective(quicksum(us) + lamb1 * (quicksum(xi_pos)+quicksum(xi_neg)) + lamb2 * quicksum( [( 1- theta) for theta in thetas]), GRB.MINIMIZE) 
    

                
    # Optimize
    m.optimize()

    # Save optimal thetas
    thetas = []
    for k in range(num_critiques+1):
        optimal_theta = m.getVarByName("theta%d" % k).X
        thetas.append(optimal_theta)
        
    critiqued_vector = np.zeros(keyphrase_freq.shape[1])
    
    # Combine weights to critiqued vector
    for c in critiques:
#         critiqued_vector[c] = 1 # set critiqued/boosted keyphrase to 1
        critiqued_vector[c] = max(keyphrase_freq[test_user , c],1)
    
    for k in range(num_critiques):
        critiqued_vector[critiques[k]] *= thetas[k+1]
    
    # Get rating score
    critique_score = predict_scores(matrix_U=reg.predict(critiqued_vector.reshape(1, -1)),
                                    matrix_V=item_latent)
    new_prediction = thetas[0]*initial_prediction_u/num_critiques + critique_score.flatten()
#     new_prediction = initial_prediction_u/num_critiques + critique_score.flatten()
#     new_prediction = critique_score.flatten()
    
    return new_prediction, thetas

# Main

In [36]:
class LP1Simplified(object):
    def __init__(self, keyphrase_freq, item_keyphrase_freq, row, matrix_Train, matrix_Test, test_users,
                 target_ranks, num_items_sampled, num_keyphrases, df,
                 max_iteration_threshold, keyphrase_popularity, dataset_name,
                 model, parameters_row, keyphrases_names, keyphrase_selection_method, max_wanted_keyphrase, lamb, **unused):
        self.keyphrase_freq = keyphrase_freq
        self.item_keyphrase_freq = item_keyphrase_freq
        self.row = row
        self.matrix_Train = matrix_Train
        self.num_users, self.num_items = matrix_Train.shape
        self.matrix_Test = matrix_Test
        self.test_users = test_users
        self.target_ranks = target_ranks
        self.num_items_sampled = num_items_sampled
        self.num_keyphrases = num_keyphrases
        self.df = df
        self.max_iteration_threshold = max_iteration_threshold
        self.keyphrase_popularity = keyphrase_popularity
        self.dataset_name = dataset_name
        self.model = model
        self.parameters_row = parameters_row
        self.keyphrase_selection_method = keyphrase_selection_method
        self.max_wanted_keyphrase = max_wanted_keyphrase
        
        self.lamb = lamb
        self.keyphrases_names = keyphrases_names

    def start_critiquing(self):
#         self.get_initial_predictions() # No need to do it every time
        self.RQ = RQ
        Yt = Y.T 
        self.Y = Y

        self.reg = reg

        self.prediction_scores = predict_scores(matrix_U=self.RQ,
                                                matrix_V=self.Y,
                                                bias=Bias).T
        
        for user in tqdm(self.test_users):
            start_time = time.time()
            # User id starts from 0
            self.row['user_id'] = user
            
            initial_prediction_items = predict_vector(rating_vector=self.prediction_scores[user],
                                                            train_vector=self.matrix_Train[user],
                                                            remove_train=True)
            # For keyphrase selection method 'diff' 
            top_recommended_keyphrase_freq = get_item_keyphrase_freq(self.item_keyphrase_freq,item = initial_prediction_items[0])
            
            # The iteration will stop if the wanted item is in top n
            for target_rank in self.target_ranks:
                self.row['target_rank'] = target_rank
                
                # Pick wanted items in test items
                candidate_items = self.matrix_Test[user].nonzero()[1]
                train_items = self.matrix_Train[user].nonzero()[1]
                wanted_items = np.setdiff1d(candidate_items, train_items)
                
                for item in wanted_items:
                    # Item id starts from 0
                    self.row['item_id'] = item
                    try:
                        self.row['item_name'] = get_restaurant_name(df_train, business_df,item)
                    except:
                        self.row['item_name'] = 'NOT_FOUND'
                    # Set the wanted item's initial rank as None
                    self.row['item_rank'] = None
                    # Set the wanted item's initial prediction score as None
                    self.row['item_score'] = None
                    
                    if self.keyphrase_selection_method == "random" or self.keyphrase_selection_method == "pop":
                        # Get the item's existing keyphrases (we can boost)
                        remaining_keyphrases = self.item_keyphrase_freq[item].nonzero()[1]
                    if self.keyphrase_selection_method == "diff":
                        # For keyphrase selection method 'diff' 
                        target_keyphrase_freq = get_item_keyphrase_freq(self.item_keyphrase_freq,item = item)
                        diff_keyphrase_freq = target_keyphrase_freq - top_recommended_keyphrase_freq
                        remaining_keyphrases = np.argsort(np.ravel(diff_keyphrase_freq))[::-1][:self.max_wanted_keyphrase]
                        
#                    print("The number of remaining_keyphrases is {}. remaining_keyphrases are: {}".format(len(remaining_keyphrases), remaining_keyphrases))
                    self.row['num_existing_keyphrases'] = len(remaining_keyphrases)
                    if len(remaining_keyphrases) == 0:
                        break
                    self.row['iteration'] = 0
                    self.row['critiqued_keyphrase'] = None
                    self.row['result'] = None
                    self.df = self.df.append(self.row, ignore_index=True)

                    query = []
                    affected_items = np.array([])
                    
                    # Set up latent embedding
                    user_latent_embedding = [Y[user]]
                    
                    for iteration in range(self.max_iteration_threshold):
                        self.row['iteration'] = iteration + 1
                                                
                        if self.keyphrase_selection_method == "pop":
                            # Always critique the most popular keyphrase
                            critiqued_keyphrase = remaining_keyphrases[np.argmax(self.keyphrase_popularity[remaining_keyphrases])]
    #                        print("remaining keyphrases popularity: {}".format(self.keyphrase_popularity[remaining_keyphrases]))
                        elif self.keyphrase_selection_method == "random":
                            critiqued_keyphrase = np.random.choice(remaining_keyphrases, size=1, replace=False)[0]
            
                        elif self.keyphrase_selection_method == "diff":
                            critiqued_keyphrase = remaining_keyphrases[0]
#                             print ('critiqued_keyphrase', critiqued_keyphrase)
                        
                        self.row['critiqued_keyphrase'] = critiqued_keyphrase
                        self.row['critiqued_keyphrase_name'] = keyphrases_names[critiqued_keyphrase]
                        query.append(critiqued_keyphrase)

                        # Get affected items (items have critiqued keyphrase)
                        current_affected_items = self.item_keyphrase_freq[:, critiqued_keyphrase].nonzero()[0]
                        affected_items = np.unique(np.concatenate((affected_items, current_affected_items))).astype(int)
                        unaffected_items = np.setdiff1d(range(self.num_items), affected_items)

                        if iteration == 0:
                            prediction_items = initial_prediction_items #calculated once for each user

                        affected_items_mask = np.in1d(prediction_items, affected_items)
                        affected_items_index_rank = np.where(affected_items_mask == True)
                        unaffected_items_index_rank = np.where(affected_items_mask == False)

                        import copy
                        
                        ## concat critique embeddings to user latent embedding
                        # Get critique vector 
                        critiqued_vector = np.zeros(self.keyphrase_freq.shape[1])
                        critiqued_vector[critiqued_keyphrase] = max(self.keyphrase_freq[user , critiqued_keyphrase],1)
                        # map user critique to user latent embedding
                        k_ci = reg.predict(critiqued_vector.reshape(1, -1)).flatten()
                        user_latent_embedding.append(k_ci)
#                         print ('user latent embedding shape: ', np.array(user_latent_embedding).shape)

                        
#                         prediction_scores_u, lambdas = Average(initial_prediction_u=self.prediction_scores[user],
#                                                                              keyphrase_freq=copy.deepcopy(self.keyphrase_freq),
#                                                                              affected_items=np.intersect1d(affected_items, prediction_items[affected_items_index_rank[0][:20]]),
#                                                                              unaffected_items=np.intersect1d(unaffected_items, prediction_items[unaffected_items_index_rank[0][:20]]),
#                                                                              num_keyphrases=self.num_keyphrases,
#                                                                              query=query,
#                                                                              test_user=user,
#                                                                              item_latent=self.RQ,
#                                                                              reg=self.reg)
#                         prediction_scores_u, lambdas = rankSVM(initial_prediction_u=self.prediction_scores[user],
#                                                                              keyphrase_freq=copy.deepcopy(self.keyphrase_freq),
#                                                                              affected_items=np.intersect1d(affected_items, prediction_items[affected_items_index_rank[0][:20]]),
#                                                                              unaffected_items=np.intersect1d(unaffected_items, prediction_items[unaffected_items_index_rank[0][:20]]),
#                                                                              num_keyphrases=self.num_keyphrases,
#                                                                              query=query,
#                                                                              test_user=user,
#                                                                              item_latent=self.RQ,
#                                                                              reg=self.reg,
#                                                                              user_latent_embedding = user_latent_embedding,
#                                                                              item_keyphrase_freq = all_item_keyphrase_freq
#                                                                              )
                        prediction_scores_u, lambdas = rankSVM2(initial_prediction_u=self.prediction_scores[user],
                                                                             keyphrase_freq=copy.deepcopy(self.keyphrase_freq),
                                                                             affected_items=np.intersect1d(affected_items, prediction_items[affected_items_index_rank[0][:20]]),
                                                                             unaffected_items=np.intersect1d(unaffected_items, prediction_items[unaffected_items_index_rank[0][:20]]),
                                                                             num_keyphrases=self.num_keyphrases,
                                                                             query=query,
                                                                             test_user=user,
                                                                             item_latent=self.RQ,
                                                                             reg=self.reg,
                                                                             user_latent_embedding = user_latent_embedding,
                                                                             item_keyphrase_freq = all_item_keyphrase_freq,
                                                                             lamb = self.lamb
                                                                             )    
                    
#                         item_keyphrase_freq = get_all_item_keyphrase_freq()
                        
                        self.row['lambda'] = lambdas
                        prediction_items = predict_vector(rating_vector=prediction_scores_u,
                                                          train_vector=self.matrix_Train[user],
                                                          remove_train=False)
                        recommended_items = prediction_items
                        
                        # Current item rank
                        item_rank = np.where(recommended_items == item)[0][0]

                        self.row['item_rank'] = item_rank
                        self.row['item_score'] = prediction_scores_u[item]

                        if item_rank + 1 <= target_rank:
                            # Items is ranked within target rank
                            self.row['result'] = 'successful'
                            self.df = self.df.append(self.row, ignore_index=True)
                            break
                        else:
                            remaining_keyphrases = np.setdiff1d(remaining_keyphrases, critiqued_keyphrase)
                            # Continue if more keyphrases and iterations remained
                            if len(remaining_keyphrases) > 0 and self.row['iteration'] < self.max_iteration_threshold:
                                self.row['result'] = None
                                self.df = self.df.append(self.row, ignore_index=True)
                            else:
                                # Otherwise, mark fail
                                self.row['result'] = 'fail'
                                self.df = self.df.append(self.row, ignore_index=True)
                                break
#                         break ## For Testing LP Objective
        
            print("User ", user ,"Elapsed: {}".format(inhour(time.time() - start_time)))
        return self.df


    def get_initial_predictions(self):
        self.RQ, Yt, Bias = plrec(self.matrix_Train,
                                       iteration=self.parameters_row['iter'],
                                       lamb=self.parameters_row['lambda'],
                                       rank=self.parameters_row['rank'])
        self.Y = Yt.T

        self.reg = LinearRegression().fit(self.keyphrase_freq, self.RQ)

        self.prediction_scores = predict_scores(matrix_U=self.RQ,
                                                matrix_V=self.Y,
                                                bias=Bias)



In [42]:
row = {}
matrix_Train = rtrain
matrix_Test = rtest
test_users = np.arange(25)
# test_users = [1]
target_ranks = [20, 50]
num_items_sampled = 5
num_keyphrases = 235
df = pd.DataFrame(row)
max_iteration_threshold = 20
keyphrase_popularity = keyphrase_popularity
dataset_name = "yelp"
model = "plrec"
parameters_row = {'iter': 10,
                  'lambda':200,
                  'rank':200}
keyphrases_names = keyphrases
keyphrase_selection_method = 'diff'
max_wanted_keyphrase = 20
lamb = [100,100]
critiquing_model = LP1Simplified(keyphrase_freq=U_K,
                                item_keyphrase_freq=I_K,
                                row=row,
                                matrix_Train=matrix_Train,
                                matrix_Test=matrix_Test,
                                test_users=test_users,
                                target_ranks=target_ranks,
                                num_items_sampled=num_items_sampled,
                                num_keyphrases=num_keyphrases,
                                df=df,
                                max_iteration_threshold=max_iteration_threshold,
                                keyphrase_popularity=keyphrase_popularity,
                                dataset_name=dataset_name,
                                model=model,
                                parameters_row=parameters_row,
                                keyphrases_names = keyphrases_names,
                                keyphrase_selection_method = keyphrase_selection_method,
                                max_wanted_keyphrase = max_wanted_keyphrase,
                                lamb = lamb)
df = critiquing_model.start_critiquing()

table_path = '../tables/critiquing/multi_step_critiquing/yelp/ranksvm/'
name = 'lamb100_100_new_prediction_method_noinitial_25user.csv'
save_dataframe_csv(df, table_path, name)

  4%|▍         | 1/25 [00:06<02:47,  6.98s/it]

User  0 Elapsed: 00:00:06


  8%|▊         | 2/25 [00:38<05:29, 14.32s/it]

User  1 Elapsed: 00:00:31


 12%|█▏        | 3/25 [00:45<04:26, 12.13s/it]

...

User  21 Elapsed: 00:00:08


 92%|█████████▏| 23/25 [03:21<00:16,  8.13s/it]

User  22 Elapsed: 00:00:03
User  23 Elapsed: 00:00:00


100%|██████████| 25/25 [03:26<00:00,  8.26s/it]

User  24 Elapsed: 00:00:05


# Tune lambda

In [44]:
lambs = [1,10,20,30,40,50,60,70,80,90,100]
for lamb1 in lambs:
    for lamb2 in lambs:
        row = {}
        matrix_Train = rtrain
        matrix_Test = rtest
        test_users = np.arange(25)
#         test_users = [1]
        target_ranks = [20, 50]
        num_items_sampled = 5
        num_keyphrases = 235
        df = pd.DataFrame(row)
        max_iteration_threshold = 20
        keyphrase_popularity = keyphrase_popularity
        dataset_name = "yelp"
        model = "plrec"
        parameters_row = {'iter': 10,
                          'lambda':200,
                          'rank':200}
        keyphrases_names = keyphrases
        keyphrase_selection_method = 'diff'
        max_wanted_keyphrase = 20
        lamb = [lamb1, lamb2]
        critiquing_model = LP1Simplified(keyphrase_freq=U_K,
                                        item_keyphrase_freq=I_K,
                                        row=row,
                                        matrix_Train=matrix_Train,
                                        matrix_Test=matrix_Test,
                                        test_users=test_users,
                                        target_ranks=target_ranks,
                                        num_items_sampled=num_items_sampled,
                                        num_keyphrases=num_keyphrases,
                                        df=df,
                                        max_iteration_threshold=max_iteration_threshold,
                                        keyphrase_popularity=keyphrase_popularity,
                                        dataset_name=dataset_name,
                                        model=model,
                                        parameters_row=parameters_row,
                                        keyphrases_names = keyphrases_names,
                                        keyphrase_selection_method = keyphrase_selection_method,
                                        max_wanted_keyphrase = max_wanted_keyphrase,
                                        lamb = lamb)
        df = critiquing_model.start_critiquing()

        table_path = '../tables/critiquing/tuning/'
        name = 'lamb1_'+ str(lamb1)+'_lamb2_' + str(lamb2) + '_test.csv'
        save_dataframe_csv(df, table_path, name)

  4%|▍         | 1/25 [00:07<02:50,  7.12s/it]

User  0 Elapsed: 00:00:07


  8%|▊         | 2/25 [00:41<05:48, 15.16s/it]

User  1 Elapsed: 00:00:33


 12%|█▏        | 3/25 [00:48<04:45, 12.99s/it]

...

User  21 Elapsed: 00:00:08


 92%|█████████▏| 23/25 [03:21<00:16,  8.17s/it]

User  22 Elapsed: 00:00:03
User  23 Elapsed: 00:00:00


100%|██████████| 25/25 [03:26<00:00,  8.26s/it]

User  24 Elapsed: 00:00:04


In [206]:
table_path = '../tables/critiquing/multi_step_critiquing/yelp/ranksvm/'
name = 'ranksvm2test.csv'
# save_dataframe_csv(df, table_path, name)
df = load_dataframe_csv(table_path,name)

In [243]:
def avg_successful_rate(df):
    num_runs = len(np.where(df['iteration'] == 0)[0])
    num_success = len(np.where(df['result'] == 'successful')[0])
    
    return num_success/num_runs
def avg_length(df,include_fail = True):
    num_runs = len(np.where(df['iteration'] == 0)[0])
    return (len(df)-num_runs)/num_runs

In [267]:
# df_5 =  df[df['target_rank'] == 5]
# df_10 = df[df['target_rank'] == 10]
df_20 = df[df['target_rank'] == 20]
df_50 = df[df['target_rank'] == 50]

In [245]:
# Ranksvm 1 50 users 20 topaffected lamb = 5
print (avg_length(df_20))
print (avg_successful_rate(df_20))
print (avg_length(df_50))
print (avg_successful_rate(df_50))

18.271084337349397
0.10090361445783133
16.94277108433735
0.1822289156626506


In [239]:
# Ranksvm2 50 users 20topaffected lamb = 5
print (avg_length(df_20))
print (avg_successful_rate(df_20))
print (avg_length(df_50))
print (avg_successful_rate(df_50))

18.370481927710845
0.09789156626506024
17.049698795180724
0.17620481927710843


In [268]:
# Ranksvm2 50 users 20topaffected lamb = 100, range = -100to100
print (avg_length(df_20))
print (avg_successful_rate(df_20))
print (avg_length(df_50))
print (avg_successful_rate(df_50))

18.13102409638554
0.11897590361445783
16.673192771084338
0.21234939759036145


In [252]:
# Avg
print (avg_length(df_20))
print (avg_successful_rate(df_20))
print (avg_length(df_50))
print (avg_successful_rate(df_50))

16.003012048192772
0.2756024096385542
14.045180722891565
0.39457831325301207


In [219]:
# Ranksvm 2 bot 20 affected
print (avg_length(df_20))
print (avg_successful_rate(df_20))
print (avg_length(df_50))
print (avg_successful_rate(df_50))

20.0
0.0
19.08888888888889
0.06666666666666667


In [290]:
# Rating obj
print (avg_length(df_20))
print (avg_successful_rate(df_20))
print (avg_length(df_50))
print (avg_successful_rate(df_50))

19.377777777777776
0.08888888888888889
18.88888888888889
0.15555555555555556


In [258]:
# top20items
print (avg_length(df_20))
print (avg_successful_rate(df_20))
print (avg_length(df_50))
print (avg_successful_rate(df_50))

18.466666666666665
0.15555555555555556
17.177777777777777
0.26666666666666666
